# Face Mask Detection Training

Importing important libraries. Proceed if you have the following modules installed already, otherwise:

```batch
py -m pip install numpy opencv-python keras sklearn tensorflow imutils
```

In [1]:
import cv2
import imutils
import h5py
import numpy as np
import warnings

from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

warnings.filterwarnings(action='ignore', category=FutureWarning)

This convolutional network utilizes a combination of two Convolutional and MaxPooling layers to extract features from the dataset. Subsequently, a Flatten layer is employed to convert the data into a one-dimensional format, followed by a Dropout layer to mitigate overfitting.

Finally, the network incorporates two Dense layers for classification purposes.

In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),

    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dropout(0.5),

    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

The generation and augmentation of the image data for training and validation.

In [3]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory('../dataset/train', batch_size=10, target_size=(150, 150))

validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory('../dataset/test', batch_size=10, target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


Initialize a callback checkpoint that saves the best model after each epoch during training.

In [4]:
checkpoint = ModelCheckpoint('../models/epoch/model-{epoch:03d}.model',
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=True,
                             mode='auto')

Actual model training and fit generation.

In [5]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10


C:\Users\natha\AppData\Local\Temp\ipykernel_8648\3620006576.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


132/132 [==============================] - ETA: 0s - loss: 0.4922 - acc: 0.7506INFO:tensorflow:Assets written to: ../models/epoch\model-001.model\assets


INFO:tensorflow:Assets written to: ../models/epoch\model-001.model\assets


132/132 [==============================] - 70s 527ms/step - loss: 0.4922 - acc: 0.7506 - val_loss: 0.2159 - val_acc: 0.9227
Epoch 2/10
132/132 [==============================] - ETA: 0s - loss: 0.2843 - acc: 0.8897INFO:tensorflow:Assets written to: ../models/epoch\model-002.model\assets


...

132/132 [==============================] - 59s 445ms/step - loss: 0.2987 - acc: 0.8821 - val_loss: 0.0989 - val_acc: 0.9639
Epoch 4/10
132/132 [==============================] - ETA: 0s - loss: 0.2542 - acc: 0.9133INFO:tensorflow:Assets written to: ../models/epoch\model-004.model\assets


INFO:tensorflow:Assets written to: ../models/epoch\model-004.model\assets


132/132 [==============================] - 64s 486ms/step - loss: 0.2542 - acc: 0.9133 - val_loss: 0.0663 - val_acc: 0.9794
Epoch 5/10
132/132 [==============================] - 64s 486ms/step - loss: 0.2615 - acc: 0.9034 - val_loss: 0.1495 - val_acc: 0.9381
Epoch 6/10
132/132 [==============================] - 65s 493ms/step - loss: 0.2277 - acc: 0.9141 - val_loss: 0.0934 - val_acc: 0.9742
Epoch 7/10
132/132 [==============================] - ETA: 0s - loss: 0.2126 - acc: 0.9232INFO:tensorflow:Assets written to: ../models/epoch\model-007.model\assets


INFO:tensorflow:Assets written to: ../models/epoch\model-007.model\assets


132/132 [==============================] - 66s 499ms/step - loss: 0.2126 - acc: 0.9232 - val_loss: 0.0290 - val_acc: 0.9948
Epoch 8/10
132/132 [==============================] - 64s 488ms/step - loss: 0.2023 - acc: 0.9262 - val_loss: 0.0738 - val_acc: 0.9639
Epoch 9/10
132/132 [==============================] - 67s 508ms/step - loss: 0.1783 - acc: 0.9376 - val_loss: 0.0422 - val_acc: 0.9897
Epoch 10/10
132/132 [==============================] - 71s 540ms/step - loss: 0.1595 - acc: 0.9346 - val_loss: 0.2257 - val_acc: 0.8918


Now, save the model to a file that will be loaded later.

In [6]:
model.save("../models/keras_model/maskup-model.keras")